In [1]:
# import libraries

import openeo
import folium
import json
import shapely.geometry
from openeo.processes import ProcessBuilder
import os


# connect to openeo
conn = openeo.connect("https://openeo.vito.be").authenticate_oidc()

# out dir
outdir = "./auxiliary_data/senales"

# aoi
aoi = json.load(open('auxiliary_data/senales/senales_wgs84.geojson'))

# define time period
time_period = ['2024-04-01', '2024-04-30']

# check the aoi
region = aoi['features'][0]['geometry']
geom = shapely.geometry.shape(region)
centroid = geom.centroid
center_latlon = [centroid.y, centroid.x]

m = folium.Map(location=center_latlon, zoom_start=10)

folium.GeoJson(aoi).add_to(m)
m

Authenticated using refresh token.


In [2]:
# load collections from openeo
s2 = conn.load_collection(
    'SENTINEL2_L1C',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=["B02", "B03", "B04", "B08", "B11"])
 
s2_L2A = conn.load_collection(
    'SENTINEL2_L2A',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=['SCL'])

worldcover = conn.load_collection(
    'ESA_WORLDCOVER_10M_2021_V2',
    spatial_extent=region,
    bands=['MAP']).resample_spatial(projection=32632)

dem = conn.load_collection(
    "COPERNICUS_30",
    spatial_extent=region,
    bands=["DEM"]).resample_spatial(projection=32632).reduce_dimension(dimension='t', reducer='mean') 
# t is reduced from dem because otherwise the dem would have several bands in t dimension. very strange

# choose water from worlcover and reduce t
water_mask = (worldcover == 80).reduce_dimension(dimension="t", reducer="mean")

# merge sentinel2s and water mask
s2 = s2.merge_cubes(s2_L2A).merge_cubes(water_mask)

# define bands
green = s2.band("B03")
swir = s2.band("B11")
nir = s2.band("B08")
scl = s2.band("SCL")
water = s2.band("MAP")

# NDSI
ndsi = (green - swir) / (green + swir)

# cloud mask
cloud_mask = ( (scl == 8) | (scl == 9) | (scl == 3) | (scl == 10) ) * 1.0 # times one forces to binary


In [3]:
valid_mask = (~cloud_mask) & (water !=1)

In [4]:
# valid_mask.download(os.path.join(outdir,'valid_mask.nc'))

In [5]:
snow_sure = (ndsi > 0.6) & (nir > 0.45) & valid_mask
no_snow_sure = (ndsi < 0) & valid_mask

In [6]:
# snow_sure.download(os.path.join(outdir,'snow_sure.nc'))
# no_snow_sure.download(os.path.join(outdir,'no_snow_sure.nc'))

In [7]:
# Combine to a snow_map: 0 = uncertain, 1 = sure no-snow, 2 = sure snow
snow_map = snow_sure.multiply(2) + no_snow_sure.multiply(1)

In [8]:
# snow_map.download(os.path.join(outdir,'snow_map.nc'))

### normalized distance

In [11]:
# add dummy bands dimension, because openEO backend wants it for some reason
snow_sure = snow_sure.add_dimension(name="bands", label="value", type="bands")

In [12]:
print(snow_sure.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-11-01T00:00:00Z', None]]}} - ['value'] - ['t', 'x', 'y', 'bands'])


In [13]:
# apply distance udf on entire polygon
distance_udf = openeo.UDF.from_file("distance_udf.py")

norm_distance = snow_sure.apply_polygon(geometries=aoi, process=distance_udf)

In [14]:
## download normalized distance
# norm_distance.download(os.path.join(outdir,'normalized_distance.nc'))

### altitude mask

In [15]:
snow_dem_combined = snow_sure.merge_cubes(dem)

In [16]:
# snow_dem_combined.download(os.path.join(outdir,'snow_dem_combined.nc'))

In [17]:
altitude_udf = openeo.UDF.from_file("altitude_mask_udf.py")

# apply altitude mask udf on entire polygon
altitude_mask = snow_dem_combined.apply_polygon(
    geometries=aoi,
    process=altitude_udf
)
# filter out the DEM that has no information
altitude_mask = altitude_mask.filter_bands(["value"])

In [18]:
# download altitude mask
# altitude_mask.download(os.path.join(outdir,'altitude_mask.nc'))

### distance index

In [19]:
# combine normalized distance and altitude mask into distance index
distance_index = norm_distance * altitude_mask

In [20]:
# distance_index.download(os.path.join(outdir,'distance_index.nc'))

In [35]:
print(distance_index.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-11-01T00:00:00Z', None]]}} - ['value'] - ['t', 'x', 'y', 'bands'])


In [50]:
scale_udf = openeo.UDF.from_file("scale_distance_udf.py")

# scale the distance values to 0-255, 255=no data
scaled_distance_index = distance_index.apply_polygon(geometries=aoi, process=scale_udf)

In [51]:
scaled_distance_index.download(os.path.join(outdir,'scaled_distance_index.nc'))